In [1]:
library(RSiena)
library(Matrix)
library(data.table)
install.packages("plyr")
library(plyr)


# Main webpage with lecture slides and sample code: https://www.stats.ox.ac.uk/~snijders/siena/
# User manual: http://www.stats.ox.ac.uk/~snijders/siena/RSiena_Manual.pdf
# Tutorial: https://www.stats.ox.ac.uk/~snijders/siena/basicRSiena.r


### Data preparation

get_edge_data <- function(data) {
  new <- data[ , c("player_id", "other_id", "count", "round")] 
  new <- new[with(new, order(round, player_id, other_id)), ]
  rownames(new) <- NULL
  return(new)
}


get_wealth_data <- function(data) {
  setDF(data) # Convert to data frame
  new <- data[ , c("player_id", "round", "wealth")] 
  rownames(new) <- NULL
  return(new)
}


get_matrix_per_wave <- function(edge_data, rnd, num_players) {
  
  wave <- edge_data[edge_data$round==rnd, ] 
  wave.copy <- wave[ , 1:3]
  wave.copy <- as.matrix(wave.copy)
  # Create empty adjacency matrix
  adj <- matrix(0, num_players, num_players)
  # Put edge values in desired places
  # "The values of the ties must be 0, 1, orNA(not available = missing); 
  # or 10 or 11 for structurally determined values""
  adj[wave.copy[ , 1:2]] <- 1 #wave.copy[ , 3]
  
  return(adj)
}


get_siena_net <- function(rawdata, t, g) {
  
  edge_data <- get_edge_data(rawdata[(rawdata$treatment==t & rawdata$group==g), ])
  
  # Groups 1, 2, and 29 are missing players and need to shift to have consecutive player ids
  players_in_game <- union(unique(edge_data$player_id), unique(edge_data$other_id))
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    # Find which player is missing - use difference between sets
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    # Decrease player numbers by one if player_id > x to account for missing player x
    edge_data$player_id[edge_data$player_id > x] <- edge_data$player_id[edge_data$player_id > x] - 1
    edge_data$other_id[edge_data$other_id > x] <- edge_data$other_id[edge_data$other_id > x] - 1
  }
  
  # Create adjacency matrix for each round
  num_rounds <- max(edge_data$round)
  matrices <- sapply( 1:num_rounds, function(rnd) get_matrix_per_wave(edge_data, rnd, num_players) )
  giving_data <- array( matrices, dim = c(num_players, num_players, num_rounds) )
  
  return(giving_data)
}


get_siena_endow <- function(rawdata, t, g) {
  # Get endowments in a column ordered by player number
  # No need to worry about missing players since the order is the same after shifting -1
  endow <- unique(rawdata[(rawdata$treatment==t & rawdata$group==g & rawdata$round==1), ])
  endow <- endow[ , c("player_id", "endowment")][order(endow$player_id), ]
  return(endow[, 2])
}


get_siena_wealth <- function(data, t, g) {
  
  # Wealth in previous period
  wealth_data <- data[(data$treatment==t & data$group==g), ]
  wealth_data <- as.data.table(wealth_data)
  wealth_data <- wealth_data[ , wealth := shift(final_wealth, fill=0), by=c('treatment', 'group_id', 'player_id')]
  
  # Take sqrt, then center by round !!!
  #wealth_data <- transform(wealth_data, wealth=sqrt(wealth))
  #wealth_data <- ddply(wealth_data, "round", transform, wealth = scale(wealth, scale=FALSE)) # scale=TRUE to get std = 1 but get Nan if all 0 in round 1
  #print(tail(wealth_data))
  # ALTERNATIVELY - Standardize by round - mean 0, std=1 !!!
  wealth_data <- ddply(wealth_data, "round", transform, wealth = scale(wealth, scale=TRUE)) 
  # Change Nan in round 1 to 0
  wealth_data$wealth[wealth_data$round==1] <- 0
  print(head(wealth_data))
  
  # Groups 1, 2, and 29 are missing players and need to shift to have consecutive player ids
  players_in_game <- wealth_data[(wealth_data$round==1), ]$player_id
  num_players <- length(players_in_game)
  if (num_players < max(players_in_game)) {
    # Find which player is missing - use difference between sets
    x <- setdiff(seq(1, max(players_in_game)), players_in_game) 
    # Decrease player numbers by one if player_id > x to account for missing player x
    wealth_data$player_id[wealth_data$player_id > x] <- wealth_data$player_id[wealth_data$player_id > x] - 1
  }
  
  wealth_data <- get_wealth_data(wealth_data)
  
  # Create marix with row per player and column per round
  num_rounds <- max(wealth_data$round)
  mat <- matrix(0, num_players, num_rounds)
  # Put wealth values in desired places
  for (i in 1:NROW(wealth_data)) {
    mat[ wealth_data[i, 1], wealth_data[i, 2] ] <- wealth_data[i, 3]
  }
  
  return(mat)
}

get_siena_data <- function(data1, data2, t, g) {
  
  # Create network wave data
  giving <- sienaDependent(get_siena_net(data1, t, g))
  # Add endowment as a constant covariate
  endowment <- coCovar(get_siena_endow(data2, t, g))
  # Add wealth as a time varying covariate - already center by round so do not center again
  wealth <- varCovar(get_siena_wealth(data2, t, g), centered=FALSE)
  
  # Put the variables together for analysis
  print(paste(t, '-', g, sep=""))
  net <- sienaDataCreate( giving, endowment, wealth )
  #print01Report(net, modelname = paste(t, '_', g, sep=""))
  return(net)
}

one_if_eq <- function(x, c){
  if (x==c) {return(1)}
  else {return(0)}
}

get_siena_data_e46 <- function(data1, data2, t, g) {
  
  # Create network wave data
  giving <- sienaDependent(get_siena_net(data1, t, g))
  # Add endowment as a constant covariate
  e <- get_siena_endow(data2, t, g)
  endowment4 <- coCovar(sapply(e, one_if_eq, c=4))
  endowment6 <- coCovar(sapply(e, one_if_eq, c=6))
  # Add wealth as a time varying covariate - already center by round so do not center again
  wealth <- varCovar(get_siena_wealth(data2, t, g), centered=FALSE)
  
  # Put the variables together for analysis
  print(paste(t, '-', g, sep=""))
  net <- sienaDataCreate( giving, endowment4, endowment6, wealth )
  #print01Report(net, modelname = paste(t, '_', g, sep=""))
  return(net)
}

get_siena_effects <- function(net, treat) {
  
  ## RECIPROCITY
  eff <- getEffects(net)
  #effectsDocumentation(eff)
  
  # Control for local bias
  # cycle3 is never significant so we will not include it
  eff <- includeEffects(eff, transTrip)
  
  ### ENDOWMENT
  
  # CONTROL for the fact that the endowed can give more
  eff <- includeEffects( eff, egoX, interaction1="endowment" )
  
  # Test for PREFERENTIAL GIVING/AVOIDANCE BY ENDOWMENT
  if (treat %in% c('E', 'EW')) {
    eff <- includeEffects( eff, altX, interaction1="endowment" )
  } else {
    eff <- includeEffects( eff, altX, interaction1="endowment", fix=TRUE )
  }
  # Alternative - diffX defined  by  the  alter-minus-ego difference of the covariate
  # over all actors to whom i has a tie - Exactly same results as altX
  # if (treat %in% c('E', 'EW)) {
  #   eff <- includeEffects( eff, diffX, interaction1 = "endowment" )
  # } else {
  #   eff <- includeEffects( eff, diffX, interaction1 = "endowment", fix=TRUE )
  # }
  
  # # ego > alter for covariate(higher), defined by the number of ties where iâs covariate is larger than alterâs, while equality counts for half
  # if (treat %in% c('E', 'EW')) {
  #   eff <- includeEffects( eff, higher, interaction1 = "endowment" )
  # } else {
  #   eff <- includeEffects( eff, higher, interaction1 = "endowment", fix=TRUE )
  # }
  
  # HOMOPHILY - no hypotheses
  # simX - 1 if same, 0 if maximum difference 4, 0.5 if difference 2
  # similar results for sameX
  #eff <- includeEffects( eff, simX, interaction1 = "endowment" )
  
  ###### Test for INEQUITY AVERSION/RECIPROCITY by endowment ######
    
  # # Interaction of reciprocity with ego's endowment:
  # eff <- includeInteraction( eff, egoX, recip, interaction1=c("endowment", "") )
  
  # Alternative specification - interaction of reciprocity with alter's endowment
  if (treat %in% c('E', 'EW')) {
    eff <- includeInteraction( eff, altX, recip, interaction1=c("endowment", "") )
  } else {
    eff <- includeInteraction( eff, altX, recip, interaction1=c("endowment", ""), fix=TRUE )
  }
  
  ## Interaction of reciprocity with similarity in endowments: N.S., high variance across groups
  # if (treat %in% c('E', 'EW')) {
  #   eff <- includeEffects( eff, simRecipX, interaction1="endowment" )
  # } else {
  #   eff <- includeEffects( eff, simRecipX, interaction1="endowment", fix=TRUE )
  # }

  # # Alternative specification - ego's endow X alter's endow X reciprpcity - N.S.
  # if (treat %in% c('E', 'EW')) {
  #   eff <- includeEffects( eff, egoXaltX, interaction1="endowment" )
  #   eff <- includeEffects( eff, egoXaltXRecip, interaction1="endowment" )
  # } else {
  #   eff <- includeEffects( eff, egoXaltX, interaction1="endowment", fix=TRUE )
  #   eff <- includeEffects( eff, egoXaltXRecip, interaction1="endowment", fix=TRUE )
  # }

  
  ### WEALTH
  
  # CONTROL for the fact that the wealthy may behave differently
  eff <- includeEffects( eff, egoX, interaction1 = "wealth" )
  
  # Test for PREFERENTIAL GIVING/AVOIDANCE BY WEALTH
  if (treat %in% c('W', 'EW')) {
    eff <- includeEffects( eff, altX, interaction1="wealth" )
  } else {
    eff <- includeEffects( eff, altX, interaction1="wealth", fix=TRUE )
  }
  # Alternative - diffX defined  by  the  alter-minus-ego difference of the covariate
  # over all actors to whom i has a tie - Exactly same results as altX!
  # if (treat %in% c('W', 'EW')) {
  #   eff <- includeEffects( eff, diffX, interaction1 = "wealth" )
  # } else {
  #   eff <- includeEffects( eff, diffX, interaction1 = "wealth", fix=TRUE )
  # }
  
  # # ego > alter for covariate(higher), defined by the number of ties where iâs covariate is larger than alterâs, while equality counts for half
  # # high variance across groups, n.s. 
  # if (treat %in% c('W', 'EW')) {
  #   eff <- includeEffects( eff, higher, interaction1 = "wealth" )
  # } else {
  #   eff <- includeEffects( eff, higher, interaction1 = "wealth", fix=TRUE )
  # }
  
  ## Test for INEQUITY AVERSION/RECIPROCITY BY WEALTH
  
  # # Alternative specification - ego's wealth X reciprocity:
  # eff <- includeInteraction( eff, egoX, recip, interaction1=c("wealth", "") )
  
  # Alternative specification - alter's wealth X reciprocity: E+ EW+
  if (treat %in% c('W', 'EW')) {
    eff <- includeInteraction( eff, altX, recip, interaction1=c("wealth", "") )
  } else {
    eff <- includeInteraction( eff, altX, recip, interaction1=c("wealth", ""), fix=TRUE )
  }
  
  ## Alternative specification - ego's wealth X alter's wealth X reciprocity - N.S.
  # if (treat %in% c('W', 'EW')) {
  #   eff <- includeEffects( eff, egoXaltX, interaction1="wealth" )
  #   eff <- includeEffects( eff, egoXaltXRecip, interaction1="wealth" )
  # } else {
  #   eff <- includeEffects( eff, egoXaltX, interaction1="wealth", fix=TRUE )
  #   eff <- includeEffects( eff, egoXaltXRecip, interaction1="wealth", fix=TRUE )
  # }
  
  ## Reciprocity X similarity in wealth: N.S., high variance across groups, n.s. 
  # if (treat %in% c('W', 'EW')) {
  #   eff <- includeEffects( eff, simRecipX, interaction1="wealth" )
  # } else {
  #   eff <- includeEffects( eff, simRecipX, interaction1="wealth", fix=TRUE )
  # }
  
  return(eff)
  
}


get_siena_effects_e46 <- function(net, treat) {
  
  ## RECIPROCITY
  eff <- getEffects(net)
  #effectsDocumentation(eff)
  
  # Control for local bias
  # cycle3 is never significant so we will not include it
  eff <- includeEffects(eff, transTrip)
  
  ### ENDOWMENT
  
  # CONTROL for the fact that the endowed can give more
  eff <- includeEffects( eff, egoX, interaction1="endowment4" )
  eff <- includeEffects( eff, egoX, interaction1="endowment6" )
  
  ## Test for PREFERENTIAL GIVING/AVOIDANCE BY ENDOWMENT
  if (treat %in% c('E', 'EW')) {
    eff <- includeEffects( eff, altX, interaction1="endowment4" )
    eff <- includeEffects( eff, altX, interaction1="endowment6" )
  } else {
    eff <- includeEffects( eff, altX, interaction1="endowment4", fix=TRUE )
    eff <- includeEffects( eff, altX, interaction1="endowment6", fix=TRUE )
  }
 
  ###### Test for INEQUITY AVERSION/RECIPROCITY by endowment ######
  
  # Interaction of reciprocity with ego's endowment:
  eff <- includeInteraction( eff, egoX, recip, interaction1=c("endowment4", "") )
  eff <- includeInteraction( eff, egoX, recip, interaction1=c("endowment6", "") )
  
  ## Alternative specification - interaction of reciprocity with alter's endowment: E+ WE-
  # if (treat %in% c('E', 'EW')) {
  #   eff <- includeInteraction( eff, altX, recip, interaction1=c("endowment", "") )
  # } else {
  #   eff <- includeInteraction( eff, altX, recip, interaction1=c("endowment", ""), fix=TRUE )
  # }
  
  
  ### WEALTH
  
  # CONTROL for the fact that the wealthy may behave differently
  eff <- includeEffects( eff, egoX, interaction1 = "wealth" )
  
  ## Test for PREFERENTIAL GIVING/AVOIDANCE BY WEALTH
  if (treat %in% c('W', 'EW')) {
    eff <- includeEffects( eff, altX, interaction1="wealth" )
  } else {
    eff <- includeEffects( eff, altX, interaction1="wealth", fix=TRUE )
  }

  ## Test for INEQUITY AVERSION/RECIPROCITY BY WEALTH
  
  # Alternative specification - ego's wealth X reciprocity:
  eff <- includeInteraction( eff, egoX, recip, interaction1=c("wealth", "") )
  
  ## Alternative specification - alter's wealth X reciprocity: E+ WE+
  # if (treat %in% c('W', 'EW')) {
  #   eff <- includeInteraction( eff, altX, recip, interaction1=c("wealth", "") )
  # } else {
  #   eff <- includeInteraction( eff, altX, recip, interaction1=c("wealth", ""), fix=TRUE )
  # }
  
  return(eff)
  
}



rawdata1 <- read.csv("data/interactions.csv", header=T)
rawdata2 <- read.csv("data/finwealth_from_interactions.csv", header=T)


#for (t in unique(rawdata1$treatment)) {
#  for (g in unique(rawdata1[(rawdata1$treatment==t), ]$group_id)) {
# for (t in c('E')) {
#   for (g in c(6)) {

### Run model for one group

temp_test <- function() {
  for (t in c('E')) {
    for (g in c(6)) {

      # 1. Get data in siena format
      net <- get_siena_data_e46(rawdata1, rawdata2, t, g)

      # 2. Include effects
      eff <- get_siena_effects_e46(net, t)

      # 3. Define the algorithm
      alg <- sienaAlgorithmCreate(projname = paste(t, '_', g, sep=""))

      # 4. Estimate the parameters and save
      ans <- siena07( alg, data = net, effects = eff)
      # Save formatted results in a html/tex file
      siena.table(ans, type="html", file=paste(t, '_', g, '.html'), sig=TRUE)

    }
  }
}

#temp_test()

get_params <- function(i, meta) {
  vec <- c(meta[[i]]$mu.ml, meta[[i]]$mu.ml.se, meta[[i]]$mu.confint, 
           meta[[i]]$sigma.ml, meta[[i]]$sigma.confint, meta[[i]]$n1)
  if (length(vec)==1) {
    print(i, vec)
    vec <- c(0, 0, 0, 0, 0, 0, 0, 0, 0, 10)
  }
  return(vec)
}

### META ANALYSIS

meta_all <- function() {
  
  #for (t in c('EW')) {
  for (t in unique(rawdata1$treatment)) {
    
    modellist <- list()
    
    #for (g in c(6, 14)) {
    for (g in unique(rawdata1[(rawdata1$treatment==t), ]$group_id)) {
      
      # 1. Get data in siena format
      net <- get_siena_data(rawdata1, rawdata2, t, g)
      
      # 2. Include effects
      eff <- get_siena_effects(net, t)
      
      # 3. Define the algorithm and estimate model
      alg <- sienaAlgorithmCreate(projname = paste(t, '_', g, sep=""))
      (model <- siena07(alg, data = net, effects = eff))
      #siena.table(model, type="html", file=paste(t, '_', g, '.html', sep=""), sig=TRUE)
      # If convergence has to be improved by a further run:
      # if (g %in% c(5)) {
      #   (model <- siena07(alg, data = net, effects = eff, prevAns=model))
      # }
      
      modellist[[paste(t, '_', g, sep="")]] <- model
      
    }
    # 4. Combine the groups and do the meta-analysis
    meta <- do.call( siena08, c( modellist, list(projname = paste("meta_", t, sep="")) ) )
    # Divert output to a text file
    summary(meta, file=TRUE, reportEstimates=True)
    
    # 5. Export summary statistics
    
    # Find number of effects in model (disregarding rate parameters)
    neff <- length(meta) - 7
    # Parameter estimates and confidence intervals
    parameters <- t(sapply(1:neff, function(i){get_params(i, meta)}))
    colnames(parameters) <- c('mu-hat', 'mu-se',
                              'mu-min', 'mu-plus', 'alpha_mu',
                              'sigma-hat', 'sigma-min', 'sigma-plus', 'alpha_sigma', 'N')
    # These are: parameter estimate for population mean; s.e.;
    # confidence interval for population mean (left, right, significance level);
    # estimate for population standard deviation;
    # confidence interval for standard deviation (left, right, significance level);
    # number of groups on which this is based.
    
    # Construct the names for the effects:
    efnames <- names(meta)[1:neff]
    # the first 5 list elements of meta correspond to the tested effects
    # The first 7 letters are redundant and will be dropped:
    efnames <- substring(efnames, 8)
    rownames(parameters) <- efnames
    print(round(parameters, 3))
    # Export
    write.csv(parameters, paste('Siena_output/params_', t, '.csv', sep=''))
    
    # All estimates
    write.csv(meta$thetadf, paste('Siena_output/thetas_', t, '.csv', sep=''))
    
  }
  
}

meta_all()



Warning message:
“no DISPLAY variable so Tk is not available”Installing package into ‘/srv/rlibs’
(as ‘lib’ is unspecified)


  treatment group_id round player_id earned_wealth       mturk_id endowment
1     Endow        6     1         1            55 A1295RHU5X5GTX         4
2     Endow        6     1         2            25 A2LVCS009DMEAT         6
3     Endow        6     1         3            60 A1IIF95II8FISQ         2
4     Endow        6     1         4            35 A3W2YF5RJ6R01N         4
5     Endow        6     1         5            15 A3K7K7IANZADVD         6
6     Endow        6     1         6            15 A2WQTNO91C05GC         2
  final_wealth wealth
1           55      0
2           25      0
3           60      0
4           35      0
5           15      0
6           15      0
[1] "Endow-6"
  effectName          include fix   test  initialValue parm
1 transitive triplets TRUE    FALSE FALSE          0   0   
  effectName    include fix   test  initialValue parm
1 endowment ego TRUE    FALSE FALSE          0   0   
  effectName      include fix  test  initialValue parm
1 endowment alter

No X11 device available, forcing use of batch mode



Start phase 0 
theta: -1.14  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -1.17506  0.05768  0.02111  0.00000  0.00949  0.00000 -0.02949  0.00000  0.00000 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.20860  0.12768  0.00913  0.00000  0.02259  0.00000 -0.08002  0.00000  0.00000 
ac 1.110 1.233 4.284 0.892 0.551 0.386 1.172 1.362 0.515 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4 Progress: 15%
the

No X11 device available, forcing use of batch mode


...


Start phase 0 
theta: -1.1  0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -1.14600 -0.00597  0.06274  0.00000  0.01514  0.00000 -0.01290  0.00000  0.00000 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.1913 -0.0286  0.1213  0.0000  0.0389  0.0000 -0.0292  0.0000  0.0000 
ac 1.049 1.603 1.573 0.808 1.494 1.114 0.737 0.688 0.845 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4 Progress: 15%
theta -1.3458 -0.0711

No X11 device available, forcing use of batch mode



Start phase 0 
theta: -1.01  0.00  0.00  0.00  0.00  0.00  0.00  0.00  0.00 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -1.04144  0.05087  0.01451  0.00000  0.00986  0.00000 -0.00635  0.00000  0.00000 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.0824  0.1356  0.0340  0.0000  0.0231  0.0000 -0.0155  0.0000  0.0000 
ac   1.109   1.631  -0.428   0.748   1.211   1.090 -10.546   0.912   0.683 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4 Progress

No X11 device available, forcing use of batch mode



Start phase 0 
theta: -0.934  0.000  0.000  0.000  0.000  0.000  0.000  0.000  0.000 

Start phase 1 
Phase 1 Iteration 1 Progress: 0%
Phase 1 Iteration 2 Progress: 0%
Phase 1 Iteration 3 Progress: 0%
Phase 1 Iteration 4 Progress: 0%
Phase 1 Iteration 5 Progress: 0%
Phase 1 Iteration 10 Progress: 0%
Phase 1 Iteration 15 Progress: 0%
Phase 1 Iteration 20 Progress: 1%
Phase 1 Iteration 25 Progress: 1%
Phase 1 Iteration 30 Progress: 1%
Phase 1 Iteration 35 Progress: 1%
Phase 1 Iteration 40 Progress: 1%
Phase 1 Iteration 45 Progress: 1%
Phase 1 Iteration 50 Progress: 2%
theta: -0.9857  0.0466  0.0331  0.0000  0.0203  0.0000 -0.0151  0.0000  0.0000 

Start phase 2.1
Phase 2 Subphase 1 Iteration 1 Progress: 15%
Phase 2 Subphase 1 Iteration 2 Progress: 15%
theta -1.0399  0.1037  0.0544  0.0000  0.0546  0.0000 -0.0418  0.0000  0.0000 
ac  1.705  3.568  0.399  1.809  1.052  1.259  1.645 -2.564  1.333 
Phase 2 Subphase 1 Iteration 3 Progress: 15%
Phase 2 Subphase 1 Iteration 4 Progress: 15%
the

In [4]:
library(gdata)  
mydata = read.csv("Siena_output/params_Endow.csv")

In [9]:
print(mydata[,1:5])

                                    X     mu.hat      mu.se      mu.min
1                 outdegree (density) -1.4177233 0.05416921 -1.53550247
2                         reciprocity  0.6929695 0.05592594  0.56676321
3                 transitive triplets  0.1047823 0.04079573  0.01990708
4                     endowment alter  0.0000000 0.00000000  0.00000000
5                       endowment ego  0.1506773 0.01437601  0.11983416
6                        wealth alter  0.0000000 0.00000000  0.00000000
7                          wealth ego -0.1422098 0.04139216 -0.23287005
8 int.  endowment alter x reciprocity  0.0000000 0.00000000  0.00000000
9    int.  wealth alter x reciprocity  0.0000000 0.00000000  0.00000000
      mu.plus
1 -1.30129465
2  0.81331201
3  0.19642988
4  0.00000000
5  0.18168820
6  0.00000000
7 -0.05338135
8  0.00000000
9  0.00000000
